# 第6章 对象引用、可变性和垃圾 回收


## 浅拷贝

- 
复制列表使用内置的类型构造函  数- 。
使用[:]进行浅    拷- 贝。
使用copy.copy函数进行  浅拷贝。

In [1]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)
l2[1] += [33, 22]
l2[2] += (10, 11)
print('l1:', l1)
print('l2:', l2)


l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


## 深拷贝

使用copy.deepcopy函数

In [2]:
# 校车乘客在途中有上有下
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)


In [3]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
# 使用浅拷贝
bus2 = copy.copy(bus1)
# 使用深拷贝
bus3 = copy.deepcopy(bus1)
bus1.drop('Bill')
bus2.passengers


['Alice', 'Claire', 'David']

In [4]:
bus3.passengers


['Alice', 'Bill', 'Claire', 'David']

## 防御可变参数

In [5]:
class TwilightBus:
    """让乘客销声匿迹的校车"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []  
        else:
            # WARN：不能直接引用变量，应使用浅拷贝
            self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)


basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
# 下车的学生从篮球队中消失了！
basketball_team


['Sue', 'Maya', 'Diana']

In [6]:
# 修改后的代码
class TwilightBus:
    """让乘客销声匿迹的校车"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []  
        else:
            # 使用构造器的浅拷贝方式
            self.passengers = list(passengers)  # 浅拷贝

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)


basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')

# 仍然保留
basketball_team


['Sue', 'Tina', 'Maya', 'Diana', 'Pat']

## del 和垃圾回收

del 语句删除名称(对象的引用)，而不是对象。
del 命令可能会导致对象被当作垃圾回收，但是仅当删除的变量保存的是对象的最后一个引用，或者无法得到对象时。重新绑定也可能会导致对象的引用数量归零，导致对象被销毁。

In [7]:
import weakref
 
 
# 1. s1 和 s2 是别名，指向同一个集合，{1, 2, 3}。
s1 = {1, 2, 3}
s2 = s1
 
 
# 2. 这个函数一定不能是要销毁的对象的绑定方法，否则会有一个指向对象的引用。
def bye():
    print('Gone ... with the wind...')
 
 
# 3. 在 s1 引用的对象上注册 bye 回调。
ender = weakref.finalize(s1, bye)
 
# 4. 调用 finalize 对象之前，.alive 属性的值为 True。
print(ender.alive)  # True
 
# 5. del 不删除对象，而是删除对象的引用。
del s1
print(ender.alive)  # True
 
# 6. 重新绑定最后一个引用 s2，让 {1, 2, 3} 无法获取。
# 对象被销毁了，调用了 bye 回调 ender.alive 的值变成了 False。
s2 = 'spam'  # Gone with the wind...
print(ender.alive)  # False

True
True
Gone ... with the wind...
False



- 平等对待所有对象：Python采用的方式是==运算符比较对象的值，而is比较引用。
- 可变性：在Python中，用户定义的类，实例默认可变。
- 对象析构和垃圾回收：CPython的垃圾回收主要依靠引用计数，在CPython2.0实现了分代垃圾回收机制，把引用循环中不可达的对象销毁。
- 参数传递（共享传参）：
- 在早期的语言中，参数传递模式有按值传递（函数得到实参的副本）和按引用传递（函数得到实参的指针）。
- 在Python中，函数得到实参的副本，但是实参始终是引用的。